In [3]:

from tdt4290.parse_lidar import parse_lidar_data
import pandas as pd
import json
import librosa
from datetime import datetime
from typing import List
import pandas as pd
from tdt4290.types_common import LidarLabeledAudio
from tdt4290.label_audio_with_lidar import label_audio_with_lidar
import librosa.display

# We'll need IPython.display's Audio widget
from IPython.display import Audio
from tdt4290 import parse_lidar, classify_lidar

In [4]:
# Check the sound file
# lidar = parse_lidar_data("./data/lidar_log_grilstad_22.09.22_10_00_13_00.txt")
# lidar_labeled_audio: List[LidarLabeledAudio] = []
# #print(lidar)
# sample_rate = 32000
# print("Some hello")
# for i, row in lidar.iterrows():
#     print("key", row)
#     enter = int(row["seconds_enter"] * sample_rate)
#     leave = int(row["seconds_leave"] * sample_rate)
#     print(enter, leave)
#     item: LidarLabeledAudio = {
#         "lidar_data": row,
#         "audio_sample": (enter, leave)
#     }
#     print("val", item)


In [5]:
sample_rate = 32000
audio_start = pd.to_datetime("2022-09-22 10:05:10.0000")
first_car_enter_audio_time = pd.Timedelta("00:00:10.608")
audio_end = pd.Timedelta("00:00:10.608")

#step 1 - Read lidar log
lidar_data = parse_lidar.parse_lidar_data("data/lidar_log_grilstad_22.09.22_10_00_13_00.txt")
#step 2 - Classify lidar
classify_lidar.classify_lidar(lidar_data)
#step 3 - link with audio
labeled = label_audio_with_lidar(
        lidar_data,
        audio_start,
        audio_length=pd.Timedelta("02:32:58")
    )
labeled

,y0,y1,height,width,front_area,datetime_enter,datetime_leave,duration,class_1,class_2,timedelta_start,timedelta_end,audio_start_index,audio_end_index
28,225,393,188,168,31584,2022-09-22 10:05:20.708,2022-09-22 10:05:21.255,0.547,medium,medium,0 days 00:00:10.708000,0 days 00:00:11.255000,319408,383408
30,147,312,185,165,30525,2022-09-22 10:05:58.322,2022-09-22 10:05:58.682,0.360,medium,medium,0 days 00:00:48.322000,0 days 00:00:48.682000,1520064,1584064
32,344,590,340,246,83640,2022-09-22 10:06:33.510,2022-09-22 10:06:35.804,2.294,large,large,0 days 00:01:23.510000,0 days 00:01:25.804000,2677024,2741024
34,362,539,145,177,25665,2022-09-22 10:06:53.551,2022-09-22 10:06:53.964,0.413,medium,medium_big,0 days 00:01:43.551000,0 days 00:01:43.964000,3288240,3352240
36,471,542,158,71,11218,2022-09-22 10:07:05.711,2022-09-22 10:07:05.831,0.120,small,small,0 days 00:01:55.711000,0 days 00:01:55.831000,3672672,3736672
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1104,370,524,162,154,24948,2022-09-22 12:37:27.857,2022-09-22 12:37:28.163,0.306,medium,medium_small,0 days 02:32:17.857000,0 days 02:32:18.163000,292384320,292448320
1106,395,560,145,165,23925,2022-09-22 12:37:34.430,2022-09-22 12:37:34.790,0.360,medium,medium,0 days 02:32:24.430000,0 days 02:32:24.790000,292595520,292659520
1108,125,278,162,153,24786,2022-09-22 12:37:36.910,2022-09-22 12:37:37.217,0.307,medium,medium_small,0 days 02:32:26.910000,0 days 02:32:27.217000,292674032,292738032
1110,120,289,152,169,25688,2022-09-22 12:37:43.110,2022-09-22 12:37:43.483,0.373,medium,medium,0 days 02:32:33.110000,0 days 02:32:33.483000,292873488,292937488


In [ ]:
# Load labeled data into file
labeled.to_csv("data/lidar_data_with_audio_indexes_1.csv")

In [6]:
sound, sr = librosa.load(".data/audio_22092022.WAV", sr=sample_rate, mono=True)
print("Sound length", len(sound))

Sound length 293696811


In [7]:
32000 * 60 * 60 * 2.5

288000000.0

In [8]:
start, end = labeled["audio_start_index"][28], labeled["audio_end_index"][28]
print(start, end)
# print(labeled["audio_start_index"][28])
# diff = end - start
# new_start = int((start + diff/2) - 1 * 32000)
# new_end = int((end - diff/2) + 1 * 32000)
# part = sound[new_start:new_end]
# start, end = 8482912, 8569056
part = sound[start:end]
Audio(data=part, rate=sample_rate)

319408 383408
